In [1]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cpu
     |████████████████████████████████| 1.6 MB 3.8 MB/s            
     |████████████████████████████████| 1.7 MB 95.4 MB/s            
     |████████████████████████████████| 186.8 MB 70 kB/s               
     |████████████████████████████████| 1.6 MB 79.1 MB/s            
     |████████████████████████████████| 5.7 MB 85.1 MB/s            
     |████████████████████████████████| 536 kB 77.2 MB/s            
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
!pip3 install transformers scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, balanced_accuracy_score
from torch.optim import Adam
from torch.utils.data import DataLoader

import evaluate
from sklearn.model_selection import train_test_split
import copy
from datasets import Dataset
from transformers import BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding

In [3]:
cd shared

/srv/shared


/opt/tljh/user/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
DEVICE = 'cpu'
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2").to(DEVICE)

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [5]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [6]:
df = pd.read_pickle('355k_13topics', compression="gzip")
X = df.text
y = df.topic 

topic_to_number = {topic: i for i, topic in enumerate(y.unique())}
y = y.map(topic_to_number)

train_texts, test_texts, train_labels, test_labels = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

train_dataset = Dataset.from_pandas(pd.DataFrame({'text': train_texts, 'label': train_labels}))
test_dataset = Dataset.from_pandas(pd.DataFrame({'text': test_texts, 'label': test_labels}))
full_dataset = Dataset.from_pandas(pd.DataFrame({'text': X, 'label': y}))

In [7]:
# создаём токенайзер и обучаем его нашему корпусу текстов
tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny2', padding=True, truncation=True)

# получаем token_ids и другие нужные атрибуты для обучения
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

train_dataset = train_dataset.map(preprocess_function)
test_dataset = test_dataset.map(preprocess_function)

Map:   0%|          | 0/283481 [00:00<?, ? examples/s]

Map:   0%|          | 0/70871 [00:00<?, ? examples/s]

In [8]:
train_dataset, test_dataset

(Dataset({
     features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 283481
 }),
 Dataset({
     features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 70871
 }))

In [9]:
def get_hf_model(model_id):
    model = BertForSequenceClassification.from_pretrained(model_id, num_labels=13)
    return model

def get_frozen_backbone(model):
    frozen_model = copy.deepcopy(model)
    
    for param in frozen_model.parameters():
        param.requires_grad = False
    for param in frozen_model.classifier.parameters():
        param.requires_grad = True
    return frozen_model

def train_model(model_id, train_dataset, test_dataset, freeze=False):
    model = get_hf_model(model_id)
    
    if freeze:
        model = get_frozen_backbone(model)
    
    training_args = TrainingArguments(
    output_dir="./model2",
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    save_total_limit=2,
    weight_decay=0.01,
    learning_rate = 3e-4,
    logging_strategy = 'epoch',
    evaluation_strategy="epoch"
    )
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator)

    trainer.train()
    
    return model

rubert = train_model('cointegrated/rubert-tiny2', train_dataset, test_dataset)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 